# 퍼셉트론

이진 분류를 하기 위한 지도학습 기반 알고리즘으로 

입력 특성에 대한 가중합이 threshold를 넘으면 1로, 넘지 않으면 -1로 분류하는 모델

In [1]:
import tensorflow as tf
import numpy as np 

이번엔 Iris 데이터 중 두 종류를 분류하는 퍼셉트론을 제작한다. y값은 1 또는 -1을 사용하고 활성화 함수로는 하이퍼탄젠트(hypertangent)함수를 사용한다.

$$\Large{ \hat{y} = tanh(w^Tx) }$$

비용 함수로는 다음 식을 사용한다. 

$$
\large{
Loss = \sum_{i=1}^N \max(0, -y_i \hat{y_i})
}
$$

실제값과 예측값이 같은 경우 (-1, -1) 또는 (1, 1)가 되므로 $ -y_i \hat{y_i}$ 는 음수가 된다. 따라서 Loss는 0이 될 것이다.

반대로 실제값과 예측값이 다른 경우((-1, 1) 또는 (1, -1))에  $ -y_i \hat{y_i}$는 양수가 되므로 Loss는 1이 될 것이다. 

따라서 해당 Loss는 실제값과 예측값이 다른 데이터의 개수로 볼 수도 있다.

In [2]:
from sklearn.datasets import load_iris
iris = load_iris()
print(iris.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

In [3]:
idx = np.in1d(iris.target, [0, 2]) # setosa와 virginica 클래스만 사용
X_data = iris.data[idx, 0:2] # feature로 sepal 컬럼만 사용
y_data = (iris.target[idx] - 1.0)[:, np.newaxis] 

In [4]:
X_data.shape, y_data.shape

((100, 2), (100, 1))

#### 이 데이터를 이용해 Perceptron을 구현해보세요! 
 - 데이터 하나당 한번씩 weights 업데이트
 - step size == 0.0003
 - iteration == 200

In [5]:
num_iter = 500
lr = 0.0003

-----

In [14]:
w_init = tf.random.normal([X_data.shape[1], 1], dtype=tf.float64)
w = tf.Variable(w_init)

b_init = tf.random.normal([1, 1], dtype=tf.float64)
b = tf.Variable(b_init)

In [15]:
w_init.shape, b_init.shape

(TensorShape([2, 1]), TensorShape([1, 1]))

In [16]:
X_data[0].shape

(2,)

shape이 맞지 않으므로 맞춰주어야함

In [9]:
X_data[0]

array([5.1, 3.5])

In [10]:
X_data[0:1] , X_data[0:1].shape

(array([[5.1, 3.5]]), (1, 2))

In [13]:
tf.expand_dims(X_data[0], 0)

<tf.Tensor: shape=(1, 2), dtype=float64, numpy=array([[5.1, 3.5]])>

In [21]:
zero = tf.constant(0, dtype=tf.float64)

for epoch in range(num_iter):
    # 데이터 하나당
    for i in range(X_data.shape[0]):
        x = X_data[i:i+1] #expand_dims를 써도 됨
        y = y_data[i:i+1]
        
        with tf.GradientTape() as tape:
            y_hat = tf.tanh(tf.matmul(x, w) + b)
            loss = tf.maximum(zero, tf.multiply(-y, y_hat))
        dw, db = tape.gradient(loss, [w, b]) 

        w.assign_sub(lr * dw) 
        b.assign_sub(lr * db) 

In [22]:
y_pred = tf.tanh(tf.matmul(X_data, w) + b)

print("예측치 :", y_pred[0].numpy(),  "정답 :", y_data[0])
print("예측치 :", y_pred[51].numpy(), "정답 :", y_data[51])
print("예측치 :", y_pred[88].numpy(), "정답 :", y_data[88])

예측치 : [-0.5008443] 정답 : [-1.]
예측치 : [0.36364475] 정답 : [1.]
예측치 : [0.22652504] 정답 : [1.]


In [23]:
print("예측치 :", -1 if y_pred[0] < 0 else 1,  "정답 :", y_data[0])
print("예측치 :", -1 if y_pred[51] < 0 else 1, "정답 :", y_data[51])
print("예측치 :", -1 if y_pred[88] < 0 else 1, "정답 :", y_data[88])

예측치 : -1 정답 : [-1.]
예측치 : 1 정답 : [1.]
예측치 : 1 정답 : [1.]


In [24]:
for epoch in range(num_iter):
    # 데이터 하나당
    for i in range(X_data.shape[0]):
        x = X_data[i:i+1] #expand_dims를 써도 됨
        y = y_data[i:i+1]
        
        with tf.GradientTape() as tape:
            y_hat = tf.tanh(tf.matmul(x, w) + b)
            loss = tf.maximum(0, tf.multiply(-y, y_hat))
        dw, db = tape.gradient(loss, [w, b]) 

        w.assign_sub(lr * dw) 
        b.assign_sub(lr * db) 

In [25]:
y_pred = tf.tanh(tf.matmul(X_data, w) + b)

print("예측치 :", -1 if y_pred[0] < 0 else 1,  "정답 :", y_data[0])
print("예측치 :", -1 if y_pred[51] < 0 else 1, "정답 :", y_data[51])
print("예측치 :", -1 if y_pred[88] < 0 else 1, "정답 :", y_data[88])

예측치 : -1 정답 : [-1.]
예측치 : 1 정답 : [1.]
예측치 : 1 정답 : [1.]


: 